# Mesh plane and line

It is sometimes necessary to get the coordinates of points, which are equally spaced on a line. Those values can then be used for sampling fields. In `discretisedfield`, this can be obtained using `line`.

`line` takes 3 input arguments:

- points `p1` and `p2` between which the line spans,
- the number of points on the line `n`.

The mesh we are going to use as an example is:

In [1]:
import discretisedfield as df

p1 = (0, 0, 0)
p2 = (100e-9, 50e-9, 20e-9)
n = (20, 10, 4)

region = df.Region(p1=p1, p2=p2)
mesh = df.Mesh(region=region, n=n)

Let us say we want to get the coordinates of 11 (`n=11`) points on the line, which spans between $(0, 0, 0)$ and $(100\,\text{nm}, 0, 20\,\text{nm})$.

In [2]:
# NBVAL_IGNORE_OUTPUT
mesh.line(p1=(0, 0, 0), p2=(100e-9, 0, 20e-9), n=11)

<generator object Mesh.line at 0x7f4e297c64a0>

This method returns a generator, which we can use as an iterator (for instance, in `for` loops). The coordinates of points are:

In [3]:
line = mesh.line(p1=(0, 0, 0), p2=(100e-9, 0, 20e-9), n=11)
list(line)

[(0.0, 0.0, 0.0),
 (1e-08, 0.0, 2e-09),
 (2e-08, 0.0, 4e-09),
 (3.0000000000000004e-08, 0.0, 6.000000000000001e-09),
 (4e-08, 0.0, 8e-09),
 (5e-08, 0.0, 1e-08),
 (6.000000000000001e-08, 0.0, 1.2000000000000002e-08),
 (7e-08, 0.0, 1.4000000000000001e-08),
 (8e-08, 0.0, 1.6e-08),
 (9e-08, 0.0, 1.8000000000000002e-08),
 (1e-07, 0.0, 2e-08)]

When asking for a mesh to give us the points on a line, both points `p1` and `p2` must belong to the mesh. For instance, if we ask for the following line:

In [4]:
p1 = (0, 0, 0)
p2 = (100e-9, 100e-9, 100e-9)

try:
    list(mesh.line(p1=p1, p2=p2, n=10))
except ValueError:
    print('Exception raised.')

Exception raised.


This is because point `p2` does not belong to the mesh region (`mesh.region`):

In [5]:
p2 in mesh.region

False

## Plane mesh

Similar to the line, we usually need to extract a plane-mesh. This method we later use for slicing fields and extracting the values of a field on the plane.

Plane mesh is obtained using `plane` method. The planes allowed must be perpendicular to x, y, or z axis. Therefore, to extract a plane, we need to define the axis to which the plane is perpendicular to as well as the point at which the plane intersects the axis. For example, we want to extract the plane perpendicular to the z-axis (in the xy-plane), which intersects it at $2.5\,\text{nm}$:

In [6]:
mesh.plane(z=2.5e-9)

Mesh(region=Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 5e-09)), n=(20, 10, 1), bc='', subregions={}, attributes={'unit': 'm', 'fourierspace': False, 'isplane': True, 'planeaxis': 2, 'point': 2.5e-09, 'axis1': 0, 'axis2': 1})

This method returns a mesh object, which consists of a single layer of cells in the z-direction and keeps the same number of cells in x and y directions:

In [7]:
mesh.plane(z=2.5e-9).n

(20, 10, 1)

In other words, the discretisation cell size in the plane mesh is the same as the size of `cell` in the original mesh. The number of cells in the x and y direction can be changed by passing `n` lenth-2 tuple of integers, which define how many cells we want in the other 2 directions.

In [8]:
mesh.plane(z=2.5e-9, n=(10, 2))

Mesh(region=Region(p1=(0.0, 0.0, 0.0), p2=(1e-07, 5e-08, 5e-09)), n=(10, 2, 1), bc='', subregions={}, attributes={'unit': 'm', 'fourierspace': False, 'isplane': True, 'planeaxis': 2, 'point': 2.5e-09, 'axis1': 0, 'axis2': 1})

The number of discretisation cells of a new mesh is:

In [9]:
mesh.plane(z=2.5e-9, n=(10, 2)).n

(10, 2, 1)

The edge lengths of the resulting mesh region is:

In [10]:
mesh.plane(z=2.5e-9, n=(10, 2)).region.edges

(1e-07, 5e-08, 5e-09)

This means that the plane mesh keeps the original dimensions in x and y directions and has the "thickness" of the original mesh discretisation cell.

Another way for asking for a plane mesh is simply by passing a string `'x'`, `'y'`, or `'z'`, without specifying the point. In that case, the mesh is sliced through the middle of the sample: 

In [11]:
plane_mesh = mesh.plane('x')

plane_mesh.region.centre

(5e-08, 2.5e-08, 1e-08)

which is the same as the centre of the original mesh:

In [12]:
mesh.region.centre

(5e-08, 2.5e-08, 1e-08)